# Create Benchmark and Statistic models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
%load_ext autoreload 
%autoreload 2
import functions as fun
#from sklearn.metrics import negative_log_likelyhood

In [2]:
# # 
# dataset = pd.read_csv('training_data.csv')
# dataset = dataset.drop(['Unnamed: 0'], axis = 1)
# dataset.plot(subplots=True, layout=(14, 4), figsize=(24, 12))
# dataset.head()

In [3]:
dataset = pd.read_csv('target.csv')
features = pd.read_csv('features.csv')
features.head()

,Unnamed: 0,epoch_utc,datetime_de,windon_forecast,windoff_forecast,solar_forecast,prices_forecast,aggregated_generation_forecast,AT_import,AT_export,...,day_of_week_mon,day_of_week_sat,day_of_week_sun,day_of_week_thu,day_of_week_tue,day_of_week_wed,hour_of_day_sin,hour_of_day_cos,hour_of_year_sin,hour_of_year_cos
0,0,1.546297e+09,2018-12-31 23:00:00+00:00,18879.75,5069.00,0.0,28.32,57054.0,61.0,4940.0,...,1,0,0,0,0,0,-0.258819,0.965926,-0.000717,1.000000
1,1,1.546301e+09,2019-01-01 00:00:00+00:00,20626.50,5042.25,0.0,10.07,57400.0,60.0,5073.0,...,0,0,0,0,1,0,0.000000,1.000000,0.000000,1.000000
2,2,1.546304e+09,2019-01-01 01:00:00+00:00,22355.50,5028.50,0.0,-4.08,57275.0,20.0,4284.0,...,0,0,0,0,1,0,0.258819,0.965926,0.000717,1.000000
3,3,1.546308e+09,2019-01-01 02:00:00+00:00,24032.50,4977.75,0.0,-9.91,58024.0,3.0,4273.0,...,0,0,0,0,1,0,0.500000,0.866025,0.001435,0.999999
4,4,1.546312e+09,2019-01-01 03:00:00+00:00,25452.00,4907.25,0.0,-7.41,59301.0,0.0,4961.0,...,0,0,0,0,1,0,0.707107,0.707107,0.002152,0.999998


In [4]:
features = features.sort_values(by='epoch_utc')

In [5]:
features = features.reset_index()
features = features.drop(columns=['index'])
features

,Unnamed: 0,epoch_utc,datetime_de,windon_forecast,windoff_forecast,solar_forecast,prices_forecast,aggregated_generation_forecast,AT_import,AT_export,...,day_of_week_mon,day_of_week_sat,day_of_week_sun,day_of_week_thu,day_of_week_tue,day_of_week_wed,hour_of_day_sin,hour_of_day_cos,hour_of_year_sin,hour_of_year_cos
0,0,1.546297e+09,2018-12-31 23:00:00+00:00,18879.75,5069.00,0.0,28.320,57054.0,61.0,4940.0,...,1,0,0,0,0,0,-0.258819,9.659258e-01,-0.000717,1.000000
1,1,1.546301e+09,2019-01-01 00:00:00+00:00,20626.50,5042.25,0.0,10.070,57400.0,60.0,5073.0,...,0,0,0,0,1,0,0.000000,1.000000e+00,0.000000,1.000000
2,2,1.546304e+09,2019-01-01 01:00:00+00:00,22355.50,5028.50,0.0,-4.080,57275.0,20.0,4284.0,...,0,0,0,0,1,0,0.258819,9.659258e-01,0.000717,1.000000
3,3,1.546308e+09,2019-01-01 02:00:00+00:00,24032.50,4977.75,0.0,-9.910,58024.0,3.0,4273.0,...,0,0,0,0,1,0,0.500000,8.660254e-01,0.001435,0.999999
4,4,1.546312e+09,2019-01-01 03:00:00+00:00,25452.00,4907.25,0.0,-7.410,59301.0,0.0,4961.0,...,0,0,0,0,1,0,0.707107,7.071068e-01,0.002152,0.999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36763,36763,1.678644e+09,2023-03-12 18:00:00+00:00,22514.75,4385.50,0.0,130.422,58176.0,0.0,2286.0,...,0,0,1,0,0,0,-1.000000,-1.836970e-16,0.938377,0.345612
36764,36764,1.678648e+09,2023-03-12 19:00:00+00:00,23578.00,4326.75,0.0,110.600,57478.0,0.0,2648.0,...,0,0,1,0,0,0,-0.965926,2.588190e-01,0.938625,0.344939
36765,36765,1.678651e+09,2023-03-12 20:00:00+00:00,24823.25,4386.00,0.0,100.600,56175.0,0.0,2591.0,...,0,0,1,0,0,0,-0.866025,5.000000e-01,0.938872,0.344266
36766,36766,1.678655e+09,2023-03-12 21:00:00+00:00,25471.50,4231.00,0.0,95.776,55808.0,0.0,3143.0,...,0,0,1,0,0,0,-0.707107,7.071068e-01,0.939119,0.343592


In [6]:
dataset.index = features.datetime_de
dataset['idx'] = features.index

In [7]:
dataset_2022 = dataset['2018-12-31 23:00:00+00:00' : '2022-12-31 23:00:00+00:00']
#features_2022 = features['2018-12-31 23:00:00+00:00' : '2022-12-31 23:00:00+00:00']

In [8]:
date =pd.to_datetime(features.datetime_de)
date.head()

0   2018-12-31 23:00:00+00:00
1   2019-01-01 00:00:00+00:00
2   2019-01-01 01:00:00+00:00
3   2019-01-01 02:00:00+00:00
4   2019-01-01 03:00:00+00:00
Name: datetime_de, dtype: datetime64[ns, UTC]

In [9]:
co2 = dataset_2022['emission_factor']
co2.index = dataset_2022.idx
#features = dataset.loc[:, [j for j in cols if j != 'emission_factor' and j in cols_features]]

In [10]:
co2.head()
#plt.plot(co2)

idx
0    298.240217
1    266.522473
2    262.594215
3    260.190772
4    250.496683
Name: emission_factor, dtype: float64

#### inspect co2 data and make train, test, val split

In [11]:
co2.describe() 

count    35065.000000
mean       475.324718
std        143.806543
min        150.310416
25%        370.647380
50%        484.372996
75%        587.043152
max        822.653306
Name: emission_factor, dtype: float64

In [12]:
train_ratio = 0.7
val_ratio = 0.9
#test_ratio = 0.1

n = len(co2)
#assert(n == len(features))
co2_train = co2[0:int(n*train_ratio)]
co2_long_train = co2[0:int(n*val_ratio)+3]
co2_val = co2[int(n*train_ratio):int(n*val_ratio)+3]
co2_test = co2[int(n*val_ratio)+3:]
date_test = date[int(n*val_ratio)+3:len(co2)]
co2_test_plot = co2_test.copy()
co2_test_plot.index = co2[:len(co2_test)].index
#assert(len(co2_train) == int(train_ratio*len(features)))

## statsistcal case
co2_stat = co2[0:len(co2)-24]
co2_stat_test = co2[len(co2)-24:]



In [13]:
assert(len(co2_stat_test) == 24)

In [14]:
co2_train.describe()

count    24545.000000
mean       455.275245
std        138.471578
min        150.882862
25%        350.452261
50%        465.222585
75%        565.721724
max        782.462741
Name: emission_factor, dtype: float64

### SARIMA

In [ ]:
import pmdarima as pm

In [ ]:
#co2_train_short = co2[0:int(0.1*len(co2))]
model_sarma_auto = pm.auto_arima(co2_train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=2, # maximum p and q
                     m=24,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # No Seasonality
                      start_P=1, start_Q=1,
                      max_P= 2, max_Q= 2,
                      D=None, 
                     trace=True,
                     error_action='ignore',  
                      suppress_warnings=True, 
                     stepwise=True)
print(model_sarma_auto.summary())

In [ ]:
forecast_period =24
config = (3,0,1)
seasonal_config = (2,0,0,24)
pred_sarma, res_sarma = sarimax_forecast(co2_long_train, forecast_period, config, seasonal_config)
pred_sarma = walk_forwad_validation(co2, co2_test, forecast_period, res_sarma, pred_sarma)
result_sarma = evaluation(co2_test, pred_sarma, method = 'Autoregression')
print(result_sarma)

#### validtaion

In [ ]:
forecast_period = int(len(co2_val) / 8)
config_sarma = (3,0,1)
seasonal_config = (2,0,0,24)
model_sarma = SARIMAX(co2_train, order = config_sarma, seasonal_order = seasonal_config)
res_sarma = model_sarma.fit()

In [ ]:
forecast_period = int(len(co2_val) / 8)

In [ ]:
pred_sarma_val,report_sarma_val,lower_sarma_val,upper_sarma_val = fun.walk_forwad_validation_arima(co2_val, forecast_period, res_sarma, alpha_low=0.025, alpha_high=0.975, n_features=0, method = 'SARMA')
print(report_sarma_val)

In [ ]:
pred_sarma_df = pd.DataFrame(pred_sarma_val)
pred_sarma_df.to_csv('sarmax_mean_wf_test.csv')
pred_sarma_df.head()
upper_val_df = pd.DataFrame(lower_sarma_val)
upper_val_df.to_csv('sarmax_upper_wf_test.csv')
upper_val_df.head()
lower_val_df = pd.DataFrame(upper_sarma_val)
lower_val_df.to_csv('sarmax_lower_wf_test.csv')
lower_val_df
date_val = date[int(n*train_ratio):int(n*val_ratio)]
date_test = date[int(n*val_ratio):n]
fun.plot_model_uncertainty2(co2_val, pred_sarma_df,lower_val_df ,upper_val_df, date_val, slice_start = '2022-01-01 23:00', slice_end =  '2022-02-15 23:00' , method = 'SARMA')

### test

In [15]:
forecast_period =24
config_sarma = (3,0,1)
seasonal_config = (2,0,0,24)
model_sarma = SARIMAX(co2_long_train, order = config_sarma, seasonal_order = seasonal_config)
res_sarma = model_sarma.fit()

In [78]:
co2_long_train.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31551, 31552, 31553, 31554, 31555, 31556, 31557, 31558, 31559,
            31560],
           dtype='int64', name='idx', length=31561)

In [79]:
len(co2_test) / 24

146.0

In [80]:
test = pd.concat([co2_long_train, co2_test], axis = 0)
test

idx
0        298.240217
1        266.522473
2        262.594215
3        260.190772
4        250.496683
            ...    
35060    195.617034
35061    196.523798
35062    199.292403
35063    194.357142
35064    150.310416
Name: emission_factor, Length: 35065, dtype: float64

In [81]:
test

idx
0        298.240217
1        266.522473
2        262.594215
3        260.190772
4        250.496683
            ...    
35060    195.617034
35061    196.523798
35062    199.292403
35063    194.357142
35064    150.310416
Name: emission_factor, Length: 35065, dtype: float64

In [82]:
assert(len(co2) == len(test))

In [16]:
pred_sarma,report_sarma,lower_sarma,upper_sarma = fun.walk_forwad_validation_arima(co2_test, forecast_period, res_sarma, alpha_low=0.025, alpha_high=0.975, n_features=0, method = 'SARMA')
print(report_sarma)

d:\ABajrami\Github\Master-Thesis-Probabilistic-Day-Ahead-Forecasting-of-the-German-CO2-Emission-Factor\statistical_models\functions.py:299: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_update = test[j:j+forecast_period]
d:\ABajrami\Github\Master-Thesis-Probabilistic-Day-Ahead-Forecasting-of-the-German-CO2-Emission-Factor\statistical_models\functions.py:299: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_update = test[j:j+forecast_period]
d:\ABajrami\Github\Master-Thes

MAE for SARMA: 65.03 g_CO2/kWh
MAPE for SARMA: 13.120000000000001 %
RMSE for SARMA: 93.69 g_CO2/kWh
Pinball for lower Quantile SARMA: 5.68 g_CO2/kWh
Pinball for higher Quantile SARMA: 6.2 g_CO2/kWh
Interval Score SARMA: 360.6758 g_CO2/kWh
Real Values insisde PI SARMA: 96.4 %
R2 SARMA: 57.720000000000006 %
Adjusted R2 SARMA: 57.720000000000006 %


In [ ]:
pred_sarma_df = pd.DataFrame(pred_sarma)
pred_sarma_df.to_csv('sarmax_mean_wf_test.csv')
pred_sarma_df.head()
upper_test_df = pd.DataFrame(lower_sarma)
upper_test_df.to_csv('sarmax_upper_wf_test.csv')
upper_test_df.head()
lower_test_df = pd.DataFrame(upper_sarma)
lower_test_df.to_csv('sarmax_lower_wf_test.csv')
lower_test_df
#date_val = date[int(n*train_ratio):int(n*val_ratio)]
#date_test = date[int(n*val_ratio):n]
fun.plot_model_uncertainty(co2_test, pred_sarma_df, lower_test_df , upper_test_df, date_test, '2022-08-01 00:00', '2022-12-15 00:00', 'SARMAX')